In [1]:
from scipy import ndimage, misc
import matplotlib.pyplot as plt
import numpy as np
import glob, os
import sys
import time

import urllib
import requests
import json
import time
import datetime

# import the necessary packages
import argparse
import datetime
import time
import cv2
%matplotlib inline

In [ ]:
#########################################
# PRE-STEP - GET NEST CAM ACCESS TOKEN  #
#########################################

import requests
import json

access_token = requests.post('https://api.home.nest.com/oauth2/access_token', data={'code':'xxxxxxx', 'client_id':'xxxxx', 'client_secret':'xxxxx', 'grant_type':'authorization_code'})
print access_token.json()

In [ ]:
#########################################
# PRE-STEP - TEST THE TOKEN             #
#########################################

import requests
import json

access_token2 = "c.xxxxxx"

result = requests.get('https://developer-api.nest.com/?auth='+access_token2)
#print (result)

#print(result.json())
result.json()["devices"]["cameras"]["xxxxx"]["snapshot_url"]

In [ ]:
#########################################
# STEP A - CAPTURE FRAMES FROM NEST CAM #
#########################################

access_token2 = "c.xxxxx"
image_count = 3600
st=0

for i in range(7000):
    
    try:
        st = str(datetime.datetime.now())
        
        #retreive the image
        result = requests.get('https://developer-api.nest.com/?auth='+access_token2)
        snapshot_url = result.json()["devices"]["cameras"]["xxxxx"]["snapshot_url"]

        #save image
        filenum = 900000+i
        filename = "camframes/5-31/"+str(filenum)+".jpg"
        urllib.urlretrieve(snapshot_url, filename)
        
    except IOError as e: 
        print ("I/O error")
        print (st)
        pass
    
    except KeyError as e2:
        #print ("KeyError (%d): %s"%(e2.errno, e2.strerror))
        print(e2)
        print("Keyerror")
        print (st)
        pass
    
    except:
        print ("other unexcepted error")
        print (st)
        pass
    
    #time_delay
    time.sleep(10)
    



In [ ]:
##########################################
# STEP B - IMAGE SUBTRACTION, DOWNSAMPLE #
##########################################

#things to do before running this file:
#  1: set input/output directory
#  4: set filenum according to the date of directory

dates = ["5-17","5-18","5-25","5-26","5-27","5-28"]
filenumidx = [517,518,525,526,527,528]

#loop through directories
for idx, date in enumerate(dates):

    # Settings
    dir = "camframes/"+str(date)+"/*.jpg"
    dir_out="camframes/class/stageB/photos3/"
    thresh_level = 15 
    filenum = filenumidx[idx]*100000 # 50900000 if may 9th, 531 if may 31st 


    # initialize variables
    prevFrame = None
    total_files = len(glob.glob(dir))
    movement_graph = [0]* total_files
    class_num = 0
    print(total_files)

    for i,file in enumerate(glob.glob(dir)):

        #grab first image
        try:
            thisFrame = misc.imread(file)
            thisFrame_gr = cv2.cvtColor(thisFrame, cv2.COLOR_BGR2GRAY)
            thisFrame_gr_blur = cv2.GaussianBlur(thisFrame_gr, (21, 21), 0)
            thisFrame_box = cv2.cvtColor(thisFrame, cv2.COLOR_BGR2GRAY)
        except:
            "file error - skipping"
            continue


        #if prevFrame is empty, then fill prevFrame and go to next loop
        if prevFrame == None:
            prevFrame = thisFrame
            prevFrame_gr_blur = thisFrame_gr_blur
            continue


        # compute diff prev & FirstFrame, max 0
        frameDelta1 = cv2.subtract(prevFrame_gr_blur, thisFrame_gr_blur)
        frameDelta2 = cv2.subtract(thisFrame_gr_blur, prevFrame_gr_blur)
        frameDelta = cv2.max(0, frameDelta1)

        # set threshold
        thresh = cv2.threshold(frameDelta, thresh_level, 255, cv2.THRESH_BINARY)[1]

        # dilate the thresholded image to fill in holes, then find contours on dilated image
        thresh = cv2.dilate(thresh, None, iterations=2)
        (cnts, _) = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        #turn into scalar
        thresh_ones = np.zeros(thresh.shape)
        thresh_ones[thresh==255] = 1
        movement_graph[i]= np.sum(thresh_ones)


        # Write images to correct directory
        if movement_graph[i] > 10000 and movement_graph[i]<200000: 
            print (file)
            plotout = 1
            misc.imsave(dir_out+str(filenum+i)+".png", thisFrame)
            #misc.imsave(dir_out+str(int(filenum+i))+"-prev.png", prevFrame)
            #misc.imsave(dir_out+str(int(filenum+i))+"-gr.png", thisFrame_gr)
            #misc.imsave(dir_out+str(int(filenum+i))+"-delta.png", frameDelta)
        else:
            plotout = 0


        #Print loop status
        sys.stdout.write('\r'+'Processing File: '+str(i)+ ' of '+ str(total_files))

        # Get ready for next loop
        prevFrame = thisFrame
        prevFrame_gr_blur = thisFrame_gr_blur

    #print(movement_graph)
    plt.plot(movement_graph)


In [ ]:
##########################################
# STEP C - MANUAL ASSIGN TO FIVE CLASSES #
##########################################

In [5]:
##########################################
# STEP D - ASSIGN TO TRAIN, VAL, TEST    #
##########################################

from scipy import ndimage, misc
import matplotlib.pyplot as plt
import numpy as np
import glob, os
import sys
import time

# import the necessary packages
import argparse
import datetime
import time
import cv2
%matplotlib inline

classdirname = ["sit", "stand", "care", "empty", "sleep"]
valratio = 2  # out of 10
testratio = 2 # out of 10
trainratio = 10-valratio-testratio

for i, classname in enumerate(classdirname):

    # Settings
    dir = "camframes/class/stageC/batch2/"+classname+"/*.png"
    dir_out_train="camframes/class/stageD/baby7/train/"+classname+"/"
    dir_out_val = "camframes/class/stageD/baby7/val/"+classname+"/"
    dir_out_test = "camframes/class/stageD/baby7/test/"+classname+"/"
    count = 0

    # initialize variables
    total_files = len(glob.glob(dir))
    print (total_files)

    for i,file in enumerate(glob.glob(dir)):

        #test for filename, extract file name 
        filename = file[(file.rfind("/")+1):]

        if "-" in filename:
            pass
            #print (filename)
        else:
            count += 1
            random = np.random.randint(1,10)
            
            if random < (trainratio+1):
                dir_out = dir_out_train
            elif random < (trainratio+valratio+1):
                dir_out = dir_out_val
            else:
                dir_out = dir_out_test
            
            thisFrame = misc.imread(file)
            misc.imsave(dir_out+str(filename), thisFrame)
        
       

1092
2070
211
1006
6129


In [18]:
##########################################
# STEP E - AUGMENT IMAGES IN TRAIN    #
##########################################

from scipy import ndimage, misc
import matplotlib.pyplot as plt
import numpy as np
import glob, os
import sys
import time

# import the necessary packages
import argparse
import datetime
import time
import cv2
%matplotlib inline

classdirname = ["sit", "stand", "care", "empty", "sleep"]
#classdirname = ["care"]
dir_in_train = "camframes/class/stageD/baby7/train/"
dir_out_train = "camframes/class/stageD/baby7aug/train/"


for j, classname in enumerate(classdirname):

    # Settings
    dir_in_train_class=dir_in_train+classname+"/"
    dir_out_train_class=dir_out_train+classname+"/"
    print(dir_in_train_class)

    # initialize variables
    count = 0
    total_files = len(glob.glob(dir_in_train_class+"*.png"))
    print (total_files)

    for i,file in enumerate(glob.glob(dir_in_train_class+"*.png")):

        try:
            img = misc.imread(file)
            hflip_img = cv2.flip(img,1)
            vflip_img = cv2.flip(img,0)
            bflip_img = cv2.flip(img,-1)
        except:
            "file error - skipping"
            continue
        
        print(file.split(".")[0].replace("baby7","baby7aug"))
        filepath = file.split(".")[0].replace("baby7","baby7aug")
        
        misc.imsave(filepath+"-hflip.png", hflip_img)
        misc.imsave(filepath+"-vflip.png", vflip_img)
        misc.imsave(filepath+"-bflip.png", bflip_img)



        #Print loop status
        sys.stdout.write('\r'+'Processing File: '+str(i)+ ' of '+ str(total_files))

            
        

camframes/class/stageD/baby7/train/sit/
176
camframes/class/stageD/baby7aug/train/sit/42200733
Processing File: 0 of 176camframes/class/stageD/baby7aug/train/sit/42200735
Processing File: 1 of 176camframes/class/stageD/baby7aug/train/sit/42200740
Processing File: 2 of 176camframes/class/stageD/baby7aug/train/sit/42200741
Processing File: 3 of 176camframes/class/stageD/baby7aug/train/sit/42200742
Processing File: 4 of 176camframes/class/stageD/baby7aug/train/sit/42200743
Processing File: 5 of 176camframes/class/stageD/baby7aug/train/sit/42200744
Processing File: 6 of 176camframes/class/stageD/baby7aug/train/sit/42200745
Processing File: 7 of 176camframes/class/stageD/baby7aug/train/sit/42200900
Processing File: 8 of 176camframes/class/stageD/baby7aug/train/sit/42300255
Processing File: 9 of 176camframes/class/stageD/baby7aug/train/sit/42300256
Processing File: 10 of 176camframes/class/stageD/baby7aug/train/sit/42300257
Processing File: 11 of 176camframes/class/stageD/baby7aug/train/sit/

In [5]:
##########################################
# CONVERT DIFFBABY TO GREYSCALE .        #
##########################################

from scipy import ndimage, misc
import matplotlib.pyplot as plt
import numpy as np
import glob, os
import sys
import time

# import the necessary packages
import argparse
import datetime
import time
import cv2
%matplotlib inline

classdirname = ["sit", "stand", "care", "empty", "sleep"]
#classdirname = ["care"]
dir_in = "camframes/class/stageD/diffbaby/"
dir_out = "camframes/class/stageD/diffbabygrey/"


for j, classname in enumerate(classdirname):

    # Settings
    dir_in_class=dir_in+classname+"/"
    dir_out_class=dir_out+classname+"/"
    print(dir_in_class)

    # initialize variables
    count = 0
    total_files = len(glob.glob(dir_in_class+"*.png"))
    print (total_files)

    for i,file in enumerate(glob.glob(dir_in_class+"*.png")):

        try:
            img = misc.imread(file)
            img_gr = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
        except:
            "file error - skipping"
            continue
        
        #print(file.split(".")[0].replace("diffbaby","diffbabygrey"))
        #filepath = file.split(".")[0].replace("diffbaby","diffbabygrey")
        
        print(dir_out+classname+"/"+str(i)+"-gr.png")
        
        misc.imsave(dir_out+classname+"/"+str(i)+"-gr.png", img_gr)




        #Print loop status
        sys.stdout.write('\r'+'Processing File: '+str(i)+ ' of '+ str(total_files))

            

camframes/class/stageD/diffbaby/sit/
43
camframes/class/stageD/diffbabygrey/sit/0-gr.png
Processing File: 0 of 43camframes/class/stageD/diffbabygrey/sit/1-gr.png
Processing File: 1 of 43camframes/class/stageD/diffbabygrey/sit/2-gr.png
Processing File: 2 of 43camframes/class/stageD/diffbabygrey/sit/3-gr.png
Processing File: 3 of 43camframes/class/stageD/diffbabygrey/sit/4-gr.png
Processing File: 4 of 43camframes/class/stageD/diffbabygrey/sit/5-gr.png
Processing File: 5 of 43camframes/class/stageD/diffbabygrey/sit/6-gr.png
Processing File: 6 of 43camframes/class/stageD/diffbabygrey/sit/7-gr.png
Processing File: 7 of 43camframes/class/stageD/diffbabygrey/sit/8-gr.png
Processing File: 8 of 43camframes/class/stageD/diffbabygrey/sit/9-gr.png
Processing File: 9 of 43camframes/class/stageD/diffbabygrey/sit/10-gr.png
Processing File: 10 of 43camframes/class/stageD/diffbabygrey/sit/11-gr.png
Processing File: 11 of 43camframes/class/stageD/diffbabygrey/sit/12-gr.png
Processing File: 12 of 43camfr